In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
#run_name = "dashing-shadow-171"
run_name = "robust-sweep-42"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
       #idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
       #high_contradiction = get_idx_with_top_column_values('contradiction_scores',n=n,ascending=False)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i+1, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['label'])[0] )
    pp_all = list(df1['pp'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp","reward_pp","reward_pp_minus_baseline", "vm_scores","acceptability_scores","sts_scores", 'lcp_conditions',
                "contradiction_scores", "pp_letter_diff",'pp_logp','ref_logp','kl_div', "reward_penalty"]
                #,
                
    #g_fields = ["pp","vm_scores"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward_pp', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats_training_step(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

def show_random_examples_for_eval_set(df, n): 
    random_idx = df.idx.drop_duplicates().sample(n).tolist()
    cols = ['epoch','pp', 'is_adv_example', 'is_valid_pp', 'reward_pp', "vm_scores", 'acceptability_scores', 'lcp_conditions',
             'sts_scores', 'contradiction_scores',  'pp_letter_diff']#
         #   ,] #'edit_distance_token_level'
    
    metric_cols = [o for o in cols if o not in ['pp', 'epoch']]
    agg_d = {k:np.mean for k in metric_cols}
    for i, idx in enumerate(random_idx): 
        print("\n###############\n")
        print("random", i+1, "\n")
        df1 = df.query("idx==@idx")
        print("Original:", pd.unique(df1['orig'])[0])
        print("Original label", pd.unique(df1['label'])[0] )
        df2 = df1[cols]
        print("Paraphrase-level view")
        df3 = df2.groupby('pp').agg({'epoch': lambda x: list(x), **agg_d})
        df3.sort_values(by ='epoch', key=lambda col: col.map(lambda x: np.min(x)), inplace=True)
        display_all(df3)
        print("Epoch-level view")
        df4 = df2.groupby('epoch').agg(agg_d)
        df4.columns = [o + "_avg" for o in df4.columns]
        display_all(df4)
    #     print("Whole table")
    #     display_all(df2)
    return 


In [ ]:
split = 'training_step'
n=3

df_d = get_training_dfs(path_run, postprocessed=False)
df = df_d[split]
if split == "training_step":
    idx_d = get_interesting_idx(df, n)
    print_interesting_text_stats_training_step(df, n)
else: 
    show_random_examples_for_eval_set(df, n)


###############

random 1 

Original: director uwe boll and the actors provide scant reason to care in this crude '70s throwback .
Original label 0
Unique paraphrases: 84
How the paraphrases change:


,,,,,,,,,,,,epoch
pp,reward_pp,reward_pp_minus_baseline,vm_scores,acceptability_scores,sts_scores,contradiction_scores,pp_letter_diff,pp_logp,ref_logp,kl_div,reward_penalty,
director uwe boll and the actors make no real sense of their relationship in this crude 60s throwback film.,0.000000,0.000000,-0.027058,0.883608,0.836669,0.177568,-15,-0.994161,-1.152963,0.230825,-0.190353,[1]
Director uwe boll and the actors provide little reason to care in this crude '70s retro.,0.000000,0.000000,-0.021770,0.685178,0.935810,0.052499,4,-0.335559,-0.660622,0.261943,-0.216014,[2]
director uwe boll and actors provide little reason to care in this crude '70s '' throwback.,0.000000,0.000000,-0.023654,0.641173,0.937837,0.027436,1,-0.378642,-0.397656,0.271948,-0.224265,[3]
director uwe boll and actors give little reason to care in this classic throwback of the 70s.,0.000000,0.000000,-0.015774,0.650096,0.932068,0.015881,-1,-0.533445,-0.776043,0.294400,-0.242781,[4]
the director uwe boll and actors provide no reason to care,0.000000,0.000000,0.033801,0.477597,0.643642,0.928045,34,-0.536306,-1.243383,0.447968,-0.369422,[5]
director uwe boll and actors have little reason to care in this crude throwback '70,0.000000,0.000000,-0.014655,0.719515,0.926254,0.022596,9,-0.390489,-0.719114,0.660798,-0.544935,[6]
film director uwe boll and actors provide nothing to care in this early 1980s throwback,0.000000,0.000000,-0.004036,0.419395,0.806993,0.727203,5,-0.948099,-1.587949,0.660834,-0.544965,[7]
director uwe boll and actors have only a brief reason to care in this 1990s '',0.000000,0.000000,0.109767,0.711814,0.667358,0.067404,14,-1.037262,-1.301694,0.757256,-0.624480,[8]
director and actors in this rough of the 1970s,0.000000,0.000000,0.505983,0.500545,0.620639,0.256580,46,-1.557755,-2.824403,0.949128,-0.782709,[9]


Best Paraphrase


,idx,epoch,orig,pp,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,label,pp_predclass,label_flip,vm_scores,sts_scores,pp_letter_diff,pp_letter_percent,contradiction_scores,acceptability_scores,pp_logp,ref_logp,diff_logp,kl_div,reward_pp,reward_pp_minus_baseline,reward_penalty,reward_pp_minus_baseline_with_penalty,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_acceptability_scores,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step
33363,6069,35,director uwe boll and the actors provide scant reason to care in this crude '70s throwback .,director uwe boll and actors have reason to care but still crude in this '70s throwback but nonetheless,0.916369,0.665824,0.665824,0,0,0,0.250545,0.92092,-11,1.119565,0.186496,0.733544,-0.563895,-2.188032,1.624137,2.281155,4.0,2.785722,-1.881181,0.904541,0.510066,0,1054,0,2.101214,0.032831,0.21875,32,32,29,32,0.714409,1.276257,0.023514,0.006686,0.008164,1.204031,0.000052,0.048042,0.011602,0.013193,0.000094,0.010281,0.00935,0.532881,0.030394,0.000011,0.000001



###############

random 2 

Original: there is so much plodding sensitivity .
Original label 0
Unique paraphrases: 75
How the paraphrases change:


...

,idx,epoch,orig,pp,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,label,pp_predclass,label_flip,vm_scores,sts_scores,pp_letter_diff,pp_letter_percent,contradiction_scores,acceptability_scores,pp_logp,ref_logp,diff_logp,kl_div,reward_pp,reward_pp_minus_baseline,reward_penalty,reward_pp_minus_baseline_with_penalty,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_acceptability_scores,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step
54615,3108,56,"assured , glossy and shot through with brittle desperation .",assured glossy and shot through with brittle desperation but but eventually,0.80587,0.238129,0.761871,1,0,1,0.567741,0.900345,-15,1.25,0.004129,0.558119,-0.512954,-2.560557,2.047603,2.336793,4.0,3.75,-1.927064,1.822936,0.935082,20,1725,1,2.220233,0.034691,0.28125,24,32,19,32,0.350155,0.631799,0.015014,0.004037,0.006424,0.578503,0.000032,0.03778,0.006018,0.010917,0.000052,0.008835,0.008541,0.343966,0.025348,0.061462,0.02962



###############

label_flips 3 

Original: it's a loathsome movie , it really is and it makes absolutely no sense .
Original label 0
Unique paraphrases: 92
How the paraphrases change:


,,,,,,,,,,,,epoch
pp,reward_pp,reward_pp_minus_baseline,vm_scores,acceptability_scores,sts_scores,contradiction_scores,pp_letter_diff,pp_logp,ref_logp,kl_div,reward_penalty,
"is a loathsome movie, and he is absolutely absurd.",0.000000,0.000000,0.027320,0.857209,0.738831,0.027867,22,-1.425276,-1.787898,0.236001,-0.194621,[1]
the movie is loathed it is and it makes absolutely no sense.,0.000000,0.000000,-0.000447,0.862601,0.932504,0.002569,12,-0.805859,-1.104392,0.263207,-0.217057,[2]
this is a terrible movie that it's very sinister and it just makes a whole lack of sense.,0.000000,0.000000,0.002952,0.917621,0.769756,0.003174,-17,-1.297166,-1.693270,0.366259,-0.302040,[3]
it's a film that's really pretty and this doesn't make sense.,0.000000,0.000000,0.007417,0.723093,0.695023,0.017334,11,-1.186578,-1.466635,0.317467,-0.261803,[4]
it's a movie that makes absolutely no sense.,0.093380,0.028893,0.005404,0.650447,0.845554,0.003078,28,-0.336592,-0.954442,0.526623,-0.434286,[5]
it's a movie that makes no sense,0.000000,0.000000,0.014415,0.667460,0.833509,0.003024,40,-0.640269,-1.610940,0.597873,-0.493043,[6]
it's a loathe movie and it makes sense,0.000000,0.000000,0.180010,0.611418,0.838579,0.774678,34,-0.662722,-1.565542,0.828852,-0.683523,[7]
it's a horrible movie and it makes absolutely no sense,0.249934,0.239842,0.014464,0.932995,0.868357,0.002475,18,-0.448237,-0.738933,1.015005,-0.837036,[8]
it's a movie that makes no sense.,0.000000,0.000000,0.004537,0.757129,0.839034,0.003170,39,-0.571316,-1.017760,0.932875,-0.769307,[9]


Best Paraphrase


,idx,epoch,orig,pp,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,label,pp_predclass,label_flip,vm_scores,sts_scores,pp_letter_diff,pp_letter_percent,contradiction_scores,acceptability_scores,pp_logp,ref_logp,diff_logp,kl_div,reward_pp,reward_pp_minus_baseline,reward_penalty,reward_pp_minus_baseline_with_penalty,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_acceptability_scores,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step
89486,5418,92,"it's a loathsome movie , it really is and it makes absolutely no sense .",it's loathsome movie really is but absolutely no sense but but nonetheless,0.938455,0.701811,0.701811,0,0,0,0.236643,0.848563,-2,1.027778,0.002593,0.708703,-0.248551,-3.334647,3.086097,2.209293,4.0,3.705644,-1.82192,1.883724,0.468201,6,2827,1,1.780048,0.027813,0.03125,32,32,33,32,0.577271,1.396974,0.024492,0.006467,0.007901,1.329416,0.000032,0.042592,0.005942,0.013282,0.000043,0.010884,0.008563,0.601198,0.03398,0.060191,0.029911


In [ ]:


import string
linking_contrast_phrases = [o.strip() for o in open("./linking_contrast_phrases.txt").readlines()]
def check_for_linking_contrast(s):
    return any([s.startswith(phrase + " ") or s.endswith(" " + phrase) for phrase in linking_contrast_phrases])

sens_cleaned = [s.strip(string.punctuation).strip().lower() for s in sens]
[check_for_linking_contrast(s) for s in sens_cleaned ]


[True, True, True, True, True, True, True, True, True, False]

In [ ]:
%%timeit
sens_cleaned = [s.strip(string.punctuation).strip().lower() for s in df.orig.drop_duplicates().tolist()]
for s in sens_cleaned: 
    if check_for_linking_contrast(s): pass#print(s)

17.1 ms ± 25 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepadre_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))


#### REMOVALS ####

not                   7
this                  5
love                  3
film                  2
movie                 2
apple                 2
like                  1
hate                  1
I do not like this    1
this apple            1
do                    1
I do not like         1
I love                1
dtype: int64

#### INSERTIONS ####

.                                    21
n't                                   7
like                                  2
that                                  2
film.                                 2
- I like it                           1
thyme                                 1
really                                1
Not so much                           1
'.                                    1
:                                     1
''.                                   1
love                                  1
ie                                    1
cherry.                               1
enjoy                          

Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)

ValueError: a must be greater than 0 unless no samples are taken